In [1]:
#on charge le dataset et on le met dans un dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('Data.csv', sep=',', low_memory=False)



In [2]:
# On ne peut pas garder toutes les colonnes, un choix s'opère donc. 

# Nous scindons les colonnes restantes dans plusieurs dataframes :

# Data, dataframe général, possédant "eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"
Data = df[["eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"]]



In [3]:
# geo, dataframe ayant pour but la géolocalisation, comprenant "eventid", "latitude", "longitude"
geo = df[["eventid", "latitude", "longitude"]].dropna()

geo.to_csv("Geo.csv")

In [4]:
#headlines, dataframe comprenant les descriptions. "eventid","summary"
headlines = df[["eventid","summary"]]

On va tester les données géographiques avec le dataframe geo

In [5]:
geo.describe()

,eventid,latitude,longitude
count,1.771340e+05,177134.000000,1.771340e+05
mean,2.002979e+11,23.498444,-4.586957e+02
std,1.321102e+09,18.569247,2.047790e+05
min,1.970000e+11,-53.154613,-8.618590e+07
25%,1.991061e+11,11.510545,4.545640e+00
50%,2.009070e+11,31.467463,4.324651e+01
75%,2.014083e+11,34.685087,6.871033e+01
max,2.017123e+11,74.633553,1.793667e+02


In [6]:
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

# créer la carte centrée sur la position moyenne des événements
map = folium.Map(location=[geo['latitude'].mean(), geo['longitude'].mean()], zoom_start=4)

# créer un groupe de marqueurs clusterisés
marker_cluster = MarkerCluster().add_to(map)

# ajouter un marqueur pour chaque événement dans le groupe de marqueurs clusterisés
for index, row in geo.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(marker_cluster)

# créer un groupe pour afficher le nombre de marqueurs dans chaque cluster
sub_group = folium.plugins.FeatureGroupSubGroup(marker_cluster, 'Events Count')

# ajouter le groupe à la carte
map.add_child(sub_group)

# ajouter un compteur pour chaque cluster
for i, layer in enumerate(marker_cluster._children.values()):
    count = len(layer._children)
    bounds = layer.get_bounds()
    pos = [(bounds[0][0] + bounds[1][0]) / 2, (bounds[0][1] + bounds[1][1]) / 2]
    folium.Marker(location=pos, icon=folium.Icon(icon='info-sign'), tooltip=str(count)).add_to(sub_group)

map.save("carte.html")

In [7]:
# On regarde le nombre de d'attentats revendiqués par rapport au nombre total 

Revendiqué = df[["claimed"]]

#On récupère le nombre de ligne ou la colonne "claimed" est égale à 1 dans la variable "count"

count = Revendiqué[Revendiqué.claimed == 1].count()

#On récupère le nombre de ligne total dans la variable "count2"

count2 = Revendiqué.count()

#On calcule le pourcentage de ligne ou la colonne "claimed" est égale à 1

pourcentage = (count/count2)*100

print(pourcentage)


claimed    16.445302
dtype: float64


Test de séparation par continents :

Très long à run

In [10]:
from geopy.geocoders import Nominatim
from tqdm import tqdm

# créer un géocodeur pour obtenir le nom du pays
geolocator = Nominatim(user_agent="geoapiExercises")

# ajouter une colonne pour le nom du pays à partir des coordonnées géographiques
def get_country(lat, lon):
    location = geolocator.reverse(f"{lat}, {lon}").raw.get('address', {})
    country = location.get('country', '')
    return country

geo['country'] = [get_country(lat, lon) for lat, lon in tqdm(zip(geo['latitude'], geo['longitude']), total=len(geo))]

# définir une fonction pour regrouper les événements par continent
def get_continent(country):
    if country == '':  # si le pays n'est pas trouvé, renvoyer une valeur nulle
        return None
    try:
        # obtenir le nom du continent à partir du nom du pays
        continent = pycountry.countries.search_fuzzy(country)[0].continent.name
        return continent
    except LookupError:
        # si le nom du pays n'est pas trouvé dans la bibliothèque pycountry, renvoyer une valeur nulle
        return None

# ajouter une colonne pour le


  0%|          | 110/177134 [00:54<24:29:50,  2.01it/s]

KeyboardInterrupt: 

In [ ]:
import pycountry

# créer un géocodeur pour obtenir le nom du pays
geolocator = Nominatim(user_agent="geoapiExercises")

# ajouter une colonne pour le nom du pays à partir des coordonnées géographiques
geo['country'] = geo.apply(lambda row: geolocator.reverse(f"{row['latitude']}, {row['longitude']}").raw.get('address', {}).get('country', ''), axis=1)

# définir une fonction pour regrouper les événements par continent
def get_continent(country):
    if country == '':  # si le pays n'est pas trouvé, renvoyer une valeur nulle
        return None
    try:
        # obtenir le nom du continent à partir du nom du pays
        continent = pycountry.countries.search_fuzzy(country)[0].continent.name
        return continent
    except LookupError:
        # si le nom du pays n'est pas trouvé dans la bibliothèque pycountry, renvoyer une valeur nulle
        return None

# ajouter une colonne pour le continent à partir du nom du pays
geo['continent'] = geo['country'].apply(get_continent)

# filtrer les lignes où le continent est défini
geo = geo.dropna(subset=['continent'])

# créer une carte pour chaque continent
continents = geo['continent'].unique()

for continent in continents:
    # filtrer les lignes pour le continent en cours
    continent_geo = geo[geo['continent'] == continent]

    # créer une carte pour le continent en cours
    m = folium.Map(location=[0, 0], zoom_start=2)

    # créer un groupe pour les marqueurs
    markers = MarkerCluster().add_to(m)

    # ajouter les marqueurs pour chaque événement
    for index, row in continent_geo.iterrows():
        folium.Marker(location=[row['latitude'], row['longitude']]).add_to(markers)

    # ajouter un bouton de bascule pour afficher/masquer les marqueurs
    feature_group = FeatureGroupSubGroup(markers, f"{continent} ({len(continent_geo)} events)")
    feature_group.add_to(m)

    # afficher la carte
    folium.LayerControl().add_to(m)
    m.save(f"{continent}.html")
